## Importing Libraries

In [ ]:
import kaggle

kaggle.api.authenticate()

path = kaggle.api.dataset_download_files('smaranjitghose/corn-or-maize-leaf-disease-dataset', path=".", unzip=True)

Dataset URL: https://www.kaggle.com/datasets/smaranjitghose/corn-or-maize-leaf-disease-dataset
None


In [7]:
import os
import shutil
from sklearn.model_selection import train_test_split

data_dir = './data'
train_dir = './train'
test_dir = './test'
val_dir = './val'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):
        images = os.listdir(class_path)
        
        train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
        train_images, val_images = train_test_split(train_images, test_size=0.2, random_state=42)

        # Ensure class subdirectories exist in train, val, and test folders
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

        # Copy images to train, val, and test directories
        for image in train_images:
            shutil.copy(os.path.join(class_path, image), os.path.join(train_dir, class_name, image))

        for image in val_images:
            shutil.copy(os.path.join(class_path, image), os.path.join(val_dir, class_name, image))

        for image in test_images:
            shutil.copy(os.path.join(class_path, image), os.path.join(test_dir, class_name, image))

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

train_data_path = "./train"
validation_data_path = "./val"
test_data_path = "./test"

training_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')

training_data = training_datagen.flow_from_directory(train_data_path, 
                                      target_size=(150, 150), 
                                      batch_size=32,
                                      class_mode='binary')  
 
print("Indices ",training_data.class_indices)

valid_datagen = ImageDataGenerator(rescale=1./255)
 
# this is a similar generator, for validation data
valid_data = valid_datagen.flow_from_directory(validation_data_path,
                                  target_size=(150,150),
                                  batch_size=32,
                                  class_mode='binary')

# Model Save Path
model_path = "saved_model.h5"
# Save only the best model
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Define Model
model = Sequential([
    Conv2D(32, kernel_size=3, input_shape=[150, 150, 3], activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(256, activation='relu'),
    Dropout(0.25),
    Dense(4, activation='softmax')  # Assuming 6 output classes
])

# Compile Model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Train Model
history = model.fit(training_data, epochs=20, verbose=1, 
          validation_data=valid_data, callbacks=callbacks_list)
# Save Final Model
model.save(model_path)
print("Training Complete. Model Saved as 'maize_pred.h5'")

# Testing
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load the test dataset
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary'
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Found 2677 images belonging to 4 classes.
Indices  {'Blight': 0, 'Common_Rust': 1, 'Gray_Leaf_Spot': 2, 'Healthy': 3}
Found 671 images belonging to 4 classes.


e:\Deep Learning\Deep_Learning_2024\Maize Disease\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
e:\Deep Learning\Deep_Learning_2024\Maize Disease\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 998ms/step - accuracy: 0.3543 - loss: 1.3244
Epoch 1: val_accuracy improved from -inf to 0.59911, saving model to saved_model.h5


84/84 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - accuracy: 0.3555 - loss: 1.3230 - val_accuracy: 0.5991 - val_loss: 0.9144
Epoch 2/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 994ms/step - accuracy: 0.7245 - loss: 0.7313
Epoch 2: val_accuracy improved from 0.59911 to 0.80030, saving model to saved_model.h5


84/84 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.7247 - loss: 0.7304 - val_accuracy: 0.8003 - val_loss: 0.4843
Epoch 3/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 652ms/step - accuracy: 0.7863 - loss: 0.5447
Epoch 3: val_accuracy improved from 0.80030 to 0.81371, saving model to saved_model.h5


...

84/84 ━━━━━━━━━━━━━━━━━━━━ 56s 670ms/step - accuracy: 0.8289 - loss: 0.4020 - val_accuracy: 0.8703 - val_loss: 0.3251
Epoch 11/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 577ms/step - accuracy: 0.8054 - loss: 0.4305
Epoch 11: val_accuracy did not improve from 0.87034
84/84 ━━━━━━━━━━━━━━━━━━━━ 52s 618ms/step - accuracy: 0.8055 - loss: 0.4304 - val_accuracy: 0.8465 - val_loss: 0.3518
Epoch 12/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 588ms/step - accuracy: 0.8322 - loss: 0.3881
Epoch 12: val_accuracy did not improve from 0.87034
84/84 ━━━━━━━━━━━━━━━━━━━━ 53s 630ms/step - accuracy: 0.8321 - loss: 0.3883 - val_accuracy: 0.8435 - val_loss: 0.3492
Epoch 13/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 587ms/step - accuracy: 0.8289 - loss: 0.3805
Epoch 13: val_accuracy did not improve from 0.87034
84/84 ━━━━━━━━━━━━━━━━━━━━ 53s 627ms/step - accuracy: 0.8290 - loss: 0.3806 - val_accuracy: 0.8554 - val_loss: 0.3204
Epoch 14/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 571ms/step - accuracy: 0.8561 - loss: 0.3623
Epoch 14: val_accuracy i

84/84 ━━━━━━━━━━━━━━━━━━━━ 52s 614ms/step - accuracy: 0.8559 - loss: 0.3628 - val_accuracy: 0.8718 - val_loss: 0.3092
Epoch 15/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 573ms/step - accuracy: 0.8433 - loss: 0.3822
Epoch 15: val_accuracy did not improve from 0.87183
84/84 ━━━━━━━━━━━━━━━━━━━━ 54s 637ms/step - accuracy: 0.8433 - loss: 0.3822 - val_accuracy: 0.8480 - val_loss: 0.3821
Epoch 16/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 631ms/step - accuracy: 0.8465 - loss: 0.3744
Epoch 16: val_accuracy did not improve from 0.87183
84/84 ━━━━━━━━━━━━━━━━━━━━ 57s 672ms/step - accuracy: 0.8465 - loss: 0.3743 - val_accuracy: 0.7556 - val_loss: 0.4952
Epoch 17/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 586ms/step - accuracy: 0.8436 - loss: 0.3663
Epoch 17: val_accuracy improved from 0.87183 to 0.87332, saving model to saved_model.h5


84/84 ━━━━━━━━━━━━━━━━━━━━ 53s 632ms/step - accuracy: 0.8436 - loss: 0.3663 - val_accuracy: 0.8733 - val_loss: 0.2935
Epoch 18/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 635ms/step - accuracy: 0.8510 - loss: 0.3550
Epoch 18: val_accuracy improved from 0.87332 to 0.88823, saving model to saved_model.h5


84/84 ━━━━━━━━━━━━━━━━━━━━ 57s 679ms/step - accuracy: 0.8510 - loss: 0.3552 - val_accuracy: 0.8882 - val_loss: 0.2857
Epoch 19/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 677ms/step - accuracy: 0.8603 - loss: 0.3270
Epoch 19: val_accuracy did not improve from 0.88823
84/84 ━━━━━━━━━━━━━━━━━━━━ 61s 718ms/step - accuracy: 0.8602 - loss: 0.3273 - val_accuracy: 0.8748 - val_loss: 0.3126
Epoch 20/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 669ms/step - accuracy: 0.8538 - loss: 0.3537
Epoch 20: val_accuracy did not improve from 0.88823
84/84 ━━━━━━━━━━━━━━━━━━━━ 60s 710ms/step - accuracy: 0.8538 - loss: 0.3537 - val_accuracy: 0.8852 - val_loss: 0.2748


Training Complete. Model Saved as 'maize_pred.h5'
Found 840 images belonging to 4 classes.
53/53 ━━━━━━━━━━━━━━━━━━━━ 12s 234ms/step - accuracy: 0.8697 - loss: 0.3393
Test Accuracy: 87.74%


In [10]:
training_data.class_indices

{'Blight': 0, 'Common_Rust': 1, 'Gray_Leaf_Spot': 2, 'Healthy': 3}